<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Заполниние-пропсусков" data-toc-modified-id="Заполниние-пропсусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Заполниние пропсусков</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Catboost" data-toc-modified-id="Catboost-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Catboost</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LGBM</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Константная-модель" data-toc-modified-id="Константная-модель-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Константная модель</a></span></li></ul></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Анализ моделей</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Описание 

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Обзор данных

In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns, numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV

import time

In [136]:
#df=pd.read_csv(r'C:\Users\Susanna\Downloads\autos.csv')
df=pd.read_csv(r'/datasets/autos.csv')

display(df.head(2))
df.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [137]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


**Вывод**  
Таблица содержит 354369 строк и 16 столбцов. Есть пропуски в столбцах VehicleType, Gearbox, Model, FuelType, NotRepaired.  
В целевом признаке Price пропусков нет.

## Подготовка данных

**Price**  
Цена колебется в интервале от 0 до 20000 евро. 
Очень много машин, цена которых не достигает и 100 евро. Это подозрительно, ведь такую машину выгоднее сдать в металлолом. 
Удалю эти строки.

In [138]:
df=df.query('Price>=100')

**RegistrationYear**    
Самый ранний год регистрации автомобиля - 1000, а самый поздний - 9999. Это явно ошибка. Оставлю только года 1920-2020.

In [139]:
df=df.query('1920<RegistrationYear<2020')

**Power**    
Мощность автмобилей колеблется от 0 до 20000 л.с. Это тоже ошибка. Оставлю только автомобили с мощностью 
4-2000 лс

In [140]:
df=df.query('4<Power<2000')
df['Power'].value_counts()#.sort_index().head(200)

75      23182
60      15354
150     14107
101     12926
140     12687
        ...  
1090        1
1968        1
1202        1
696         1
1394        1
Name: Power, Length: 595, dtype: int64

### Заполниние пропсусков

Оценю кол-во пропусков:

In [141]:
print('В VehicleType пропусков {:.2%}'.format(df['VehicleType'].isna().sum()/len(df)))
print('В Gearbox пропусков {:.2%}'.format(df['Gearbox'].isna().sum()/len(df)))
print('В Model пропусков {:.2%}'.format(df['Model'].isna().sum()/len(df)))
print('В FuelType пропусков {:.2%}'.format(df['FuelType'].isna().sum()/len(df)))
print('В NotRepaired пропусков {:.2%}'.format(df['NotRepaired'].isna().sum()/len(df)))

В VehicleType пропусков 6.89%
В Gearbox пропусков 1.95%
В Model пропусков 4.02%
В FuelType пропусков 6.38%
В NotRepaired пропусков 15.25%


**VehicleType**

In [142]:
df['VehicleType'].value_counts(dropna=False)

sedan          82917
small          69734
wagon          59151
bus            26178
NaN            21067
convertible    18750
coupe          14564
suv            10946
other           2465
Name: VehicleType, dtype: int64

Пропусков мало, но думаю что можно их перевести в наиболее популярный класс - автомобилей седан.

In [143]:
def vehicle(row):
    if row['VehicleType']!=row['VehicleType']:
        return 'sedan'
    else:
        return row['VehicleType']
df['VehicleType']=df.apply(vehicle, axis=1)

df['VehicleType'].value_counts(dropna=False)

sedan          103984
small           69734
wagon           59151
bus             26178
convertible     18750
coupe           14564
suv             10946
other            2465
Name: VehicleType, dtype: int64

**Gearbox**

In [144]:
df['Gearbox'].value_counts(dropna=False)

manual    239602
auto       60217
NaN         5953
Name: Gearbox, dtype: int64

Пропусков мало, но думаю что можно их перевести в наиболее популярный класс - автомобилей с ручной коробкой передач.

In [145]:
def gearbox(row):
    if row['Gearbox']!=row['Gearbox']:
        return 'manual'
    else:
        return row['Gearbox']
df['Gearbox']=df.apply(gearbox, axis=1)

df['Gearbox'].value_counts(dropna=False)

manual    245555
auto       60217
Name: Gearbox, dtype: int64

**Model**

In [146]:
df['Model'].value_counts(dropna=False)

golf                  25916
other                 20721
3er                   17722
NaN                   12297
polo                  11078
                      ...  
kalina                    5
i3                        5
rangerover                3
serie_3                   3
range_rover_evoque        2
Name: Model, Length: 250, dtype: int64

4% не так много, я удалю строки с попусщенной моделью

In [147]:
df.dropna(subset=['Model'], inplace=True)
df['Model'].isna().sum()

0

**FuelType**

In [148]:
df['FuelType'].value_counts(dropna=False)

petrol      184658
gasoline     87326
NaN          16189
lpg           4509
cng            481
hybrid         181
other           72
electric        59
Name: FuelType, dtype: int64

6.38% отнесу к самому популярному классу бензиновых

In [149]:
def fuel(row):
    if row['FuelType']!=row['FuelType']:
        return 'petrol'
    else:
        return row['FuelType']
df['FuelType']=df.apply(fuel, axis=1)

df['FuelType'].value_counts(dropna=False)

petrol      200847
gasoline     87326
lpg           4509
cng            481
hybrid         181
other           72
electric        59
Name: FuelType, dtype: int64

**NotRepaired**

In [150]:
df['NotRepaired'].value_counts(dropna=False)

no     223254
NaN     42413
yes     27808
Name: NotRepaired, dtype: int64

Предпологаю, что если про ремонт вообще ничего не указанно то его скорее всего его и не было.

In [151]:
def repair(row):
    if row['NotRepaired']!=row['NotRepaired']:
        return 'no'
    else:
        return row['NotRepaired']
df['NotRepaired']=df.apply(repair, axis=1)

df['NotRepaired'].value_counts(dropna=False)

no     265667
yes     27808
Name: NotRepaired, dtype: int64

Проверю как изсенилась таблица:

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293475 entries, 2 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        293475 non-null  object
 1   Price              293475 non-null  int64 
 2   VehicleType        293475 non-null  object
 3   RegistrationYear   293475 non-null  int64 
 4   Gearbox            293475 non-null  object
 5   Power              293475 non-null  int64 
 6   Model              293475 non-null  object
 7   Kilometer          293475 non-null  int64 
 8   RegistrationMonth  293475 non-null  int64 
 9   FuelType           293475 non-null  object
 10  Brand              293475 non-null  object
 11  NotRepaired        293475 non-null  object
 12  DateCreated        293475 non-null  object
 13  NumberOfPictures   293475 non-null  int64 
 14  PostalCode         293475 non-null  int64 
 15  LastSeen           293475 non-null  object
dtypes: int64(7), object(

удалю не нужные для обучения моделей столбцы

In [153]:
df_model=df.drop(['DateCrawled', 'RegistrationMonth', 'DateCreated', 'PostalCode', 'LastSeen'], axis=1)

In [154]:
for i in df_model.columns:
    print(i)
    print(df_model[i].value_counts())
    print()

Price
1500     4382
500      4326
1200     3733
2500     3652
1000     3589
         ... 
10895       1
7049        1
11919       1
14220       1
2691        1
Name: Price, Length: 3534, dtype: int64

VehicleType
sedan          98687
small          67339
wagon          57540
bus            25467
convertible    18059
coupe          13835
suv            10338
other           2210
Name: VehicleType, dtype: int64

RegistrationYear
1999    18694
2005    18277
2006    18123
2000    17784
2003    17413
        ...  
1932        1
1931        1
1949        1
1928        1
1927        1
Name: RegistrationYear, Length: 87, dtype: int64

Gearbox
manual    235749
auto       57726
Name: Gearbox, dtype: int64

Power
75      22113
60      14633
150     13544
101     12436
140     12337
        ...  
1240        1
1241        1
421         1
808         1
1362        1
Name: Power, Length: 574, dtype: int64

Model
golf                  25916
other                 20721
3er                   17722
polo

In [155]:
df_model=df_model.drop(['NumberOfPictures'], axis=1)

In [156]:
df_model=df_model.drop_duplicates()
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245062 entries, 2 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             245062 non-null  int64 
 1   VehicleType       245062 non-null  object
 2   RegistrationYear  245062 non-null  int64 
 3   Gearbox           245062 non-null  object
 4   Power             245062 non-null  int64 
 5   Model             245062 non-null  object
 6   Kilometer         245062 non-null  int64 
 7   FuelType          245062 non-null  object
 8   Brand             245062 non-null  object
 9   NotRepaired       245062 non-null  object
dtypes: int64(4), object(6)
memory usage: 20.6+ MB


**Вывод**    
Заполнила все пропуски, удалила некоректные строки.  
Также удалила столбцы, которые на мой взгляд не влияют на стоимость автомобилей ('DateCrawled', 'RegistrationMonth', 'DateCreated', 'PostalCode', 'LastSeen', 'NumberOfPictures').  
Удлила дубликаты.


## Обучение моделей

In [157]:
features_train, features_valid, target_train, target_valid = train_test_split(
    df_model.drop('Price', axis=1), df_model.Price, test_size=0.25, random_state=12345)

In [158]:
features_train.shape, features_valid.shape, target_train.shape, target_valid.shape

((183796, 9), (61266, 9), (183796,), (61266,))

In [159]:
table=pd.DataFrame()

In [160]:
cat_features=['VehicleType', "Gearbox", "Model", "FuelType","Brand","NotRepaired"]

### Catboost

In [161]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=50)

In [162]:
model = CatBoostRegressor()
params = {'learning_rate' : [0.1, 0.15], 'depth': [1,5]}

grid_search = GridSearchCV(model, params, scoring="neg_mean_squared_error", cv = 3)
grid_search.fit(features_train, target_train, cat_features=cat_features, verbose=200)

0:	learn: 4527.1236521	total: 17.2ms	remaining: 17.2s
200:	learn: 2306.7238641	total: 1.73s	remaining: 6.89s
400:	learn: 2239.7784867	total: 3.37s	remaining: 5.03s
600:	learn: 2223.3443984	total: 5s	remaining: 3.32s
800:	learn: 2214.5201062	total: 6.63s	remaining: 1.65s
999:	learn: 2208.6920567	total: 8.35s	remaining: 0us
0:	learn: 4521.7376951	total: 18.4ms	remaining: 18.4s
200:	learn: 2300.7781336	total: 1.8s	remaining: 7.18s
400:	learn: 2233.4586644	total: 3.61s	remaining: 5.39s
600:	learn: 2216.7666635	total: 5.63s	remaining: 3.73s
800:	learn: 2207.6307758	total: 7.31s	remaining: 1.82s
999:	learn: 2201.4331334	total: 9.04s	remaining: 0us
0:	learn: 4525.4280812	total: 18.8ms	remaining: 18.7s
200:	learn: 2301.5394653	total: 2.28s	remaining: 9.07s
400:	learn: 2234.0125709	total: 4.17s	remaining: 6.24s
600:	learn: 2215.1397652	total: 6.17s	remaining: 4.1s
800:	learn: 2205.7516779	total: 7.9s	remaining: 1.96s
999:	learn: 2200.0008862	total: 9.6s	remaining: 0us
0:	learn: 4458.0812905	tot

GridSearchCV(cv=3, error_score=nan,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001E43FEA5608>,
             iid='deprecated', n_jobs=None,
             param_grid={'depth': [1, 5], 'learning_rate': [0.1, 0.15]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [163]:
grid_search.best_params_

{'depth': 5, 'learning_rate': 0.15}

In [165]:
model=grid_search.best_estimator_

In [166]:
start=time.time()
model.fit(features_train, target_train, cat_features=cat_features, verbose=200) ### время обучения
table.loc['Catboost', 'fit_time']=time.time()-start

0:	learn: 4257.5743127	total: 87.7ms	remaining: 1m 27s
200:	learn: 1710.1990681	total: 16.3s	remaining: 1m 4s
400:	learn: 1655.1791599	total: 32.9s	remaining: 49.2s
600:	learn: 1624.6204248	total: 49s	remaining: 32.5s
800:	learn: 1602.7937615	total: 1m 6s	remaining: 16.5s
999:	learn: 1586.4474810	total: 1m 23s	remaining: 0us


In [167]:
start=time.time()
predictions = model.predict(features_valid) ### время предсказания
table.loc['Catboost', 'predict_time']=time.time()-start

In [168]:
rmse=mean_squared_error(target_valid, predictions)**0.5
table.loc['Catboost', 'rmse']=rmse
rmse

1635.460531590162

### LGBM

Порядковое кодирование категориальных признаков

In [169]:
df_model=df_model.reset_index(drop=True)

from sklearn.preprocessing import OrdinalEncoder
encoder=OrdinalEncoder()
data_ordinal=pd.DataFrame(encoder.fit_transform(df_model.drop(columns=['Price', 'RegistrationYear', 'Power', 'Kilometer'])),
                          columns=df_model.drop(columns=['Price', 'RegistrationYear', 'Power', 'Kilometer']).columns)

data_ordinal=pd.concat([data_ordinal, df_model.loc[:, ['Price', 'RegistrationYear', 'Power', 'Kilometer']]], axis=1)
data_ordinal

,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired,Price,RegistrationYear,Power,Kilometer
0,6.0,0.0,117.0,2.0,14.0,0.0,9800,2004,163,125000
1,5.0,1.0,116.0,6.0,37.0,0.0,1500,2001,75,150000
2,5.0,1.0,101.0,2.0,31.0,0.0,3600,2008,69,90000
3,4.0,1.0,11.0,6.0,2.0,1.0,650,1995,102,150000
4,1.0,1.0,8.0,6.0,25.0,0.0,2200,2004,109,150000
...,...,...,...,...,...,...,...,...,...,...
245057,4.0,0.0,4.0,6.0,0.0,0.0,5250,2016,150,150000
245058,4.0,1.0,140.0,6.0,30.0,1.0,3200,2004,225,150000
245059,1.0,0.0,106.0,6.0,32.0,0.0,1199,2000,101,125000
245060,0.0,1.0,223.0,2.0,37.0,0.0,9200,1996,102,150000


In [170]:
features = data_ordinal.drop('Price', axis=1)
target = data_ordinal['Price']

In [171]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.25, random_state = 12345)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((183796, 9), (61266, 9), (183796,), (61266,))

In [172]:
model = LGBMRegressor(random_state=12345)
params = {"max_bin": [300,500],
              "learning_rate" : [0.01,0.05],
              "num_leaves": [150, 300]}
grid_search = GridSearchCV(model, n_jobs=-1, param_grid=params, cv = 3, scoring="neg_mean_squared_error", verbose=5)
grid_search.fit(x_train, y_train, categorical_feature =cat_features)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  14 out of  24 | elapsed:   20.3s remaining:   14.5s
[Parallel(n_jobs=-1)]: Done  19 out of  24 | elapsed:   27.5s remaining:    7.2s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   29.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   29.9s finished
C:\Users\Susanna\anaconda3\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


GridSearchCV(cv=3, error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=12345,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05], 'max_bin': [300, 500],
                         'num_leaves': [150, 300]},
             pre_dispatch='2*n_jobs', refit=True, 

In [174]:
grid_search.best_params_

{'learning_rate': 0.05, 'max_bin': 500, 'num_leaves': 300}

In [175]:
model=grid_search.best_estimator_

In [176]:
start=time.time()
model.fit(x_train, y_train, verbose=5)
table.loc['Lgbm', 'fit_time']=time.time()-start

In [177]:
start=time.time()
predictions = model.predict(x_test)
table.loc['Lgbm', 'predict_time']=time.time()-start

In [178]:
rmse=mean_squared_error(y_test, predictions)**0.5
table.loc['Lgbm', 'rmse']=rmse
rmse

1650.8190647837712

### Случайный лес

In [179]:
model = RandomForestRegressor(random_state=12345)
params = {"max_depth": (1,5), "n_estimators":(1,100)}
grid_search = GridSearchCV(model, n_jobs=-1, param_grid=params, cv = 3, scoring="neg_mean_squared_error")
grid_search.fit(x_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False,
                                             random_state=12345, verbose=0,
                                             warm_sta

In [181]:
grid_search.best_params_

{'max_depth': 5, 'n_estimators': 100}

In [182]:
model=grid_search.best_estimator_

In [183]:
start=time.time()
model.fit(x_train, y_train)
table.loc['RandomForest', 'fit_time']=time.time()-start

In [184]:
start=time.time()
predictions = model.predict(x_test)
table.loc['RandomForest', 'predict_time']=time.time()-start

In [185]:
rmse=mean_squared_error(y_test, predictions)**0.5
table.loc['RandomForest', 'rmse']=rmse
rmse

2501.946417652129

### Константная модель

In [186]:
a=target_train.median() ### верно, константу следует вычислять по обучающей выборке
constant=[a]*target_valid.shape[0]
rmse=mean_squared_error(target_valid, constant)**0.5 ### должна быть тестовая выборка
table.loc['Constant_model', 'rmse']=rmse
rmse

4946.516611349963

**Вывод**    
Использвала 3 модели: Catboost, LightGMB и RandomForest.  
Выбрала оптимальные гиперпараметры, посчитала время обучения и время предсказания моделей.  

## Анализ моделей

In [187]:
table

,fit_time,predict_time,rmse
Catboost,84.428158,0.205821,1635.460532
Lgbm,2.667866,0.179520,1650.819065
RandomForest,14.248872,0.273942,2501.946418
Constant_model,NaN,NaN,4946.516611


## Общий вывод    
Я использвала 3 модели: Catboost, LightGMB и RandomForest.  
Все три показали результат лучше, чем константая модель.  
RandomForest показала худший реузльтат rmse=2502.   
У Lighgbm и Catboost результат похожий: rmse=1635 и 1651 соответственно.   
При этом RandomForest обучается и предсказывает гороздо дольше, чем другие модели.   
Самой быстрообучаемой моделью оказалась Lighgbm.  

Заказчику советую модель LightGBM, так как она показала наилучший результат и по скорости и по качеству.